Hello World! Feel Free to play around for testing.

In [1]:
import pandas as pd
import numpy as np

# Step 1: Data Loading
# Load the members' information from the CSV file
claims_df = pd.read_csv('../data/raw/Claims.csv')

hello World


In [ ]:
claims_df